In [140]:
import robotic as ry
import numpy as np
import time
print('version:', ry.__version__, ry.compiled())

version: 0.2.2 compile time: Nov 26 2024 16:47:22


In [141]:
C = ry.Config()
C.addFile("maze.g")
C.addFile("cargo2.g")
#C.view()

In [142]:
ego = C.getFrame("cargo2")

end_pos = C.getFrame("goal_area").getPosition()
start_pos = C.getFrame("cargo2").getPosition()

q_start = ego.getJointState()
print("q_start: ",q_start)

ego.setPosition(end_pos)
q_end = ego.getJointState()

print("q_end: ",q_end)


ego.setPosition(start_pos)
C.view()

q_start:  [0.5 1. ]
q_end:  [-4.5 -3. ]


0

In [143]:
#RRT Part

In [144]:
# rrt function, returns the pathways.
def rrt_path(q_start, q_end):
    for i in range(5):
        rrt = ry.PathFinder()
        rrt.setProblem(C, [q_start], [q_end])
        result = rrt.solve()
        print("result.feasible: ",result.feasible)
        if result.feasible:
            return result.x
    raise ValueError("No feasible path found after 5 attempts. Please restart the kernel")

In [145]:
#viewing the solution
path_to_goal = rrt_path(q_start, q_end)

#for config in path_to_goal:
#    C.setJointState(config)
#    C.view()
#    time.sleep(0.1)

result.feasible:  True


In [146]:
#print(path_to_goal) # rrt returns an array of points
del C

In [147]:
# reconfiguration
C = ry.Config()
C.addFile("maze.g")
C.addFile("cargo.g")
C.addFile("ego.g")
C.view()

0

In [148]:
# PART 2    
path_to_goal_3d = []
for i, point in enumerate(path_to_goal):
    # Add a z-coordinate of 0.0 to each point
    path_to_goal_3d.append([point[0], point[1], np.float64(0.0)])
    f = C.addFrame(f"waypoint{i}")
    f.setPosition(path_to_goal_3d[i])
print(path_to_goal_3d)

[[np.float64(0.5), np.float64(1.0), np.float64(0.0)], [np.float64(0.5509817952456437), np.float64(0.9139717688573614), np.float64(0.0)], [np.float64(0.4713550620564835), np.float64(0.8534772785163206), np.float64(0.0)], [np.float64(0.3918269102229651), np.float64(0.7928532480832914), np.float64(0.0)], [np.float64(0.3104114489687117), np.float64(0.7347884337052435), np.float64(0.0)], [np.float64(0.21840231422199552), np.float64(0.6956180479123945), np.float64(0.0)], [np.float64(0.13968118423065545), np.float64(0.6339497116863656), np.float64(0.0)], [np.float64(0.1270835882890223), np.float64(0.5347463822236336), np.float64(0.0)], [np.float64(0.04697135130250377), np.float64(0.4748963238693104), np.float64(0.0)], [np.float64(-0.03302587639011775), np.float64(0.4148926276373709), np.float64(0.0)], [np.float64(-0.11314022361328975), np.float64(0.35504539404211577), np.float64(0.0)], [np.float64(-0.162116187433036), np.float64(0.26785964424361425), np.float64(0.0)], [np.float64(-0.216047483

In [168]:
# for i in range(len(path_to_goal)): # for each waypoint in rrt
komo = ry.KOMO(C, phases=3, slicesPerPhase=30, kOrder=2, enableCollisions=True)  
    
komo.addControlObjective([], 0)
komo.addControlObjective([], 2)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)

komo.addObjective([1], ry.FS.distance, ["ego1","cargo"],ry.OT.eq)
komo.addModeSwitch([1,-1], ry.SY.stable, ["ego1", "cargo"], True)

# solve
ret = ry.NLP_Solver(komo.nlp(), verbose=0).solve() 

print(ret)
print(ret.feasible)
komo.view_play(True, f"Solution, {ret.feasible}, {ret.eq}")
q = komo.getPath()
print(q.shape)

{ time: 0.035005, evals: 7, done: 1, feasible: 0, eq: 0.10931, ineq: 0, sos: 17.1969, f: 0 }
False
(90, 4)


In [169]:
R = komo.report(False, plotOverTime=True)

In [121]:
# 1 komo, we use Stable all the time. and 
total_path = []
current_holder = None

C_segment1 = C
segment_path = []

#for i in range(len(path_to_goal)): # for each waypoint in rrt
komo_ego1 = ry.KOMO(C_segment1, phases=3, slicesPerPhase=30, kOrder=2, enableCollisions=True)  
    
komo_ego1.addControlObjective([], 0)
komo_ego1.addControlObjective([], 2)
komo_ego1.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)

if current_holder != "ego1": # if the current holder is not ego1
    komo_ego1.addObjective([1], ry.FS.distance, ["ego1","cargo"],ry.OT.eq)
    komo_ego1.addModeSwitch([1.4,-1], ry.SY.stable, ["ego1", "cargo"], True)
else: # already holding
    komo_ego1.addModeSwitch([1,-1], ry.SY.stable, ["ego1", "cargo"], True)
    
komo_ego1.addObjective([2], ry.FS.position, ["cargo"], ry.OT.eq, [1e2], path_to_goal_3d[2])

# solve
ret1 = ry.NLP_Solver(komo_ego1.nlp(), verbose=0).solve() 
cost1 = ret1.eq



C_segment2 = C

#for i in range(len(path_to_goal)): # for each waypoint in rrt
komo_ego2 = ry.KOMO(C_segment2, phases=3, slicesPerPhase=30, kOrder=2, enableCollisions=True)  
    
komo_ego2.addControlObjective([], 0)
komo_ego2.addControlObjective([], 2)
komo_ego2.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)

if current_holder != "ego2": # if the current holder is not ego1
    komo_ego2.addObjective([1], ry.FS.distance, ["ego2","cargo"],ry.OT.eq)
    komo_ego2.addModeSwitch([1.4,-1], ry.SY.stable, ["ego2", "cargo"], True)
else: # already holding
    komo_ego2.addModeSwitch([1,-1], ry.SY.stable, ["ego2", "cargo"], True)
    
komo_ego2.addObjective([2], ry.FS.position, ["cargo"], ry.OT.eq, [1e2], path_to_goal_3d[2])

# solve
ret2 = ry.NLP_Solver(komo_ego2.nlp(), verbose=0).solve() 
cost2 = ret2.eq


print("cost1:",cost1, "cost2:",cost2, "ret1.feasible:",ret1.feasible,"ret2.feasible:",ret2.feasible)

if cost1 < cost2:
    segment_path = komo_ego1.getPath()
    current_holder = "ego1"
    C = C_segment1
else:
    segment_path = komo_ego2.getPath()
    current_holder = "ego2"
    C = C_segment2

C.view()

C.setJointState(segment_path[-1])
total_path.extend(segment_path)
    
# Visualize the entire movement
for i, config in enumerate(total_path):
    C.setJointState(config)
    C.view(False, f"Step {i}")
    time.sleep(0.1)
R = komo_ego1.report(False, plotOverTime=True)

cost1: 11.21938716653611 cost2: 11.872308231773733 ret1.feasible: False ret2.feasible: False


In [108]:
print(ret)
print(ret.feasible)
komo.view_play(True, f"Solution, {ret.feasible}, {ret.eq}")
q = komo.getPath()
print(q.shape)

{ time: 0.291095, evals: 95, done: 1, feasible: 0, eq: 13.6088, ineq: 0, sos: 16.2845, f: 0 }
False
STACK9 /usr/bin/python3(+0x198a6b) [0x5bd41c3a5a6b]
STACK8 /usr/bin/python3(_PyObject_MakeTpCall
STACK7 /usr/bin/python3(+0x18ae12) [0x5bd41c397e12]
STACK6 /home/omer/.local/lib/python3.10/site-packages/robotic/_robotic.so(+0x3f21e) [0x7b3ca0f2721e]
STACK5 /home/omer/.local/lib/python3.10/site-packages/robotic/_robotic.so(+0x8f6ac) [0x7b3ca0f776ac]
STACK4 KOMO::view_play(bool, char const*, double, char const*)
STACK3 KOMO::view(bool, char const*)
STACK2 KOMO::info_objectiveErrorTraces()
STACK1 rai::Array<double>::operator()(int) const
STACK0 rai::LogToken::~LogToken()


== ERROR:array.ipp:operator():738(-2) CHECK failed: 'nd==1 && (uint)i<d0' -- 1D range error (1=1, 480<480)


RuntimeError: array.ipp:operator():738(-2) CHECK failed: 'nd==1 && (uint)i<d0' -- 1D range error (1=1, 480<480)

In [82]:
R = komo.report(False, plotOverTime=True)